# ═══════════════════════════════════════════════════════════════════
# 🏛️ TLA Dataset Preparation for Egyptian Transliteration RAG System
# ═══════════════════════════════════════════════════════════════════

## 📦 Part 1: Install & Import Libraries

In [44]:
# 🔹 Upgrade pip first
!pip install --upgrade pip --quiet

# 🔹 Core Dependencies
!pip install datasets>=2.18.0 --quiet
!pip install transformers>=4.38.0 --quiet
!pip install torch>=2.2.0 --quiet
!pip install pandas>=2.2.0 --quiet
!pip install numpy>=1.26.0 --quiet

# 🔹 Translation
!pip install sentencepiece>=0.2.0 --quiet

# 🔹 Vector Database
!pip install qdrant-client>=1.7.0 --quiet

# 🔹 Ollama Cloud API
!pip install httpx>=0.25.2,<0.26.0 --quiet
!pip install ollama>=0.1.7 --quiet

# 🔹 BM25 for Hybrid Search
!pip install rank-bm25>=0.2.2 --quiet

# 🔹 Utilities
!pip install tqdm>=4.66.0 --quiet
!pip install python-dotenv>=1.0.0 --quiet
!pip install jupyter>=1.0.0 --quiet
!pip install ipywidgets>=8.1.0 --quiet
!pip install scikit-learn --quiet
!pip install matplotlib --quiet
!pip install sentence-transformers --quiet
!pip install nltk==3.9.2 --quiet
!pip install rouge-score==0.1.2 --quiet
!pip install sacrebleu==2.6.0 --quiet


/bin/bash: line 1: 0.26.0: No such file or directory


In [45]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [46]:
import pandas as pd
import numpy as np
import re
import unicodedata
from tqdm.auto import tqdm
from datasets import load_dataset
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import subprocess
import json
import ollama

print("✅ All imports successful!")


✅ All imports successful!


In [47]:
## configuration
# Models
EMBEDDING_MODEL = "bge-m3:latest"  # Ollama local

# Settings
VECTOR_DIM = 1024
TRAIN_SPLIT = 0.99  # 95% for training, 5% for testing

# Egyptian character mapping (uniliteral signs)
EGYPTIAN_CHAR_MAP = {
    # Traditional → Normalized
    'ꜣ': 'a',      # vulture (aleph)
    'ꞽ': 'i',      # reed (yodh)
    'y': 'y',      # double yodh
    'ꜥ': 'a',      # arm (ayin)
    'w': 'w',      # quail
    'b': 'b',      # leg
    'p': 'p',      # stool
    'f': 'f',      # viper
    'm': 'm',      # owl
    'n': 'n',      # water
    'r': 'r',      # mouth
    'h': 'h',      # shelter
    'ḥ': 'h',      # wick
    'ḫ': 'kh',     # placenta
    'ẖ': 'kh',     # belly
    's': 's',      # cloth
    'š': 'sh',     # pool
    'ḳ': 'q',      # hill
    'q': 'q',      # hill
    'k': 'k',      # basket
    'g': 'g',      # stand
    't': 't',      # bun
    'ṯ': 'tj',     # rope
    'd': 'd',      # hand
    'ḏ': 'dj',     # cobra

    # Additional special characters
    'ṭ': 't',
    'ḍ': 'd',
    'ṣ': 's',
    'ẓ': 'z',
    'ḥ': 'h',
}

# Suffixes to remove (pronouns and particles)
SUFFIXES_TO_REMOVE = [
    '=f',   # his/him
    '=k',   # your/you (masc)
    '=ṯ',   # your/you (fem)
    '=s',   # her/it
    '=sn',  # their/them
    '=ꞽ',   # my/me
    '=n',   # our/us
    '=tn',  # your/you (pl)
    '=fꞽ',  # variant
]

print(f"🔧 Configuration loaded")
print(f"   Training split: {TRAIN_SPLIT*100}%")
print(f"   Embedding model: {EMBEDDING_MODEL}")

🔧 Configuration loaded
   Training split: 99.0%
   Embedding model: bge-m3:latest


In [48]:
## part 3: Load dataset
print("📥 Loading TLA dataset from HuggingFace...")

dataset = load_dataset(
    "thesaurus-linguae-aegyptiae/tla-Earlier_Egyptian_original-v18-premium",
    split="train"
)

df = pd.DataFrame(dataset)

print(f"✅ Loaded {len(df)} records")
print(f"\nColumns: {list(df.columns)}")
print(f"\nSample record:")
print(df.iloc[0][['transliteration', 'translation', 'UPOS']].to_dict())

📥 Loading TLA dataset from HuggingFace...
✅ Loaded 12773 records

Columns: ['hieroglyphs', 'transliteration', 'lemmatization', 'UPOS', 'glossing', 'translation', 'dateNotBefore', 'dateNotAfter']

Sample record:
{'transliteration': 'nḏ (w)di̯ r =s', 'translation': '(es) werde zerrieben, (es) werde darauf gelegt.', 'UPOS': 'VERB VERB ADP PRON'}


In [49]:
## part 4 : Data Cleaning
print("\n" + "="*70)
print("🧹 STEP 1: Removing unwanted columns")
print("="*70)

# Remove unwanted columns
columns_to_drop = ['hieroglyphs', 'dateNotBefore', 'dateNotAfter']
df_clean = df.drop(columns=columns_to_drop)

print(f"✅ Removed columns: {columns_to_drop}")
print(f"   Remaining columns: {list(df_clean.columns)}")

# Remove rows with missing critical data
print("\n🧹 STEP 2: Removing rows with missing data")
initial_count = len(df_clean)

df_clean = df_clean.dropna(subset=['transliteration', 'translation'])
df_clean = df_clean[df_clean['transliteration'].str.strip() != '']
df_clean = df_clean[df_clean['translation'].str.strip() != '']

print(f"✅ Removed {initial_count - len(df_clean)} rows with missing data")
print(f"   Records remaining: {len(df_clean)}")

# Remove duplicates
print("\n🧹 STEP 3: Removing duplicates")
initial_count = len(df_clean)

df_clean = df_clean.drop_duplicates(subset=['transliteration'], keep='first')

print(f"✅ Removed {initial_count - len(df_clean)} duplicate records")
print(f"   Unique records: {len(df_clean)}")

df_clean = df_clean.reset_index(drop=True)


🧹 STEP 1: Removing unwanted columns
✅ Removed columns: ['hieroglyphs', 'dateNotBefore', 'dateNotAfter']
   Remaining columns: ['transliteration', 'lemmatization', 'UPOS', 'glossing', 'translation']

🧹 STEP 2: Removing rows with missing data
✅ Removed 0 rows with missing data
   Records remaining: 12773

🧹 STEP 3: Removing duplicates
✅ Removed 3685 duplicate records
   Unique records: 9088


In [50]:
## part 5: Transliteration Normalization
def normalize_transliteration(text):
    """
    Normalize Egyptian transliteration:
    1. Remove brackets
    2. Lowercase
    3. Map special characters
    4. Remove suffixes
    5. Clean spaces
    """
    if not isinstance(text, str) or text.strip() == '':
        return ""

    # Step 1: Remove brackets (but keep content)
    text = re.sub(r'[()]', '', text)

    # Step 2: Normalize Unicode (NFC form)
    text = unicodedata.normalize('NFC', text)

    # REMOVE combining marks (important for di̯, etc.)
    text = ''.join(c for c in text if not unicodedata.combining(c))

    # Step 3: Lowercase
    text = text.lower()

    # Step 4: Map Egyptian characters
    for egy_char, normalized in EGYPTIAN_CHAR_MAP.items():
        text = text.replace(egy_char.lower(), normalized)

    # Step 5: Remove suffixes (pronouns/particles)
    for suffix in SUFFIXES_TO_REMOVE:
        # Match suffix at word boundaries or before spaces/dots
        pattern = re.escape(suffix) + r'(?=[\s\.]|$)'
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)

    # Step 6: Clean up extra spaces and dots
    text = re.sub(r'\.+', '.', text)  # Multiple dots to single
    text = re.sub(r'\s+', ' ', text)  # Multiple spaces to single
    text = text.strip('. ')  # Remove leading/trailing dots and spaces

    return text

print("\n" + "="*70)
print("🔤 STEP 4: Normalizing transliterations")
print("="*70)

# Test normalization on sample
sample_text = df_clean.iloc[0]['transliteration']
normalized_sample = normalize_transliteration(sample_text)

print(f"\n📝 Sample normalization:")
print(f"   Original:   {sample_text}")
print(f"   Normalized: {normalized_sample}")

# Apply normalization to entire dataset
print(f"\n🔄 Normalizing {len(df_clean)} transliterations...")

df_clean['transliteration_normalized'] = df_clean['transliteration'].apply(
    normalize_transliteration
)


# Remove empty normalizations
df_clean = df_clean[df_clean['transliteration_normalized'].str.len() > 0]
df_clean = df_clean.reset_index(drop=True)

print(f"✅ Normalization complete!")
print(f"   Valid records: {len(df_clean)}")

# Show more examples
print(f"\n📋 Sample normalizations:")
for i in range(min(5, len(df_clean))):
    orig = df_clean.iloc[i]['transliteration']
    norm = df_clean.iloc[i]['transliteration_normalized']
    print(f"   {i+1}. {orig[:40]:40} → {norm[:40]}")



🔤 STEP 4: Normalizing transliterations

📝 Sample normalization:
   Original:   nḏ (w)di̯ r =s
   Normalized: ndj wdi r

🔄 Normalizing 9088 transliterations...
✅ Normalization complete!
   Valid records: 9088

📋 Sample normalizations:
   1. nḏ (w)di̯ r =s                           → ndj wdi r
   2. n ṯw ꞽm =sn                              → n tjw im
   3. ḫꜣ m tʾ ḥnq.t kꜣ(.PL) ꜣpd(.PL) n ꞽmꜣḫ ꞽm → kha m ta hnq.t ka.pl apd.pl n imakh im.i
   4. ꜥḥꜥ                                      → ꜥhꜥ
   5. (w)sꞽr wnꞽs m n =k ꞽr.t-ḥr.w ꞽꜥb n =k s( → wsir wnis m n ir.t-hr.w iꜥb n si ir ra


In [51]:
## part 6: Train/ test split

print("\n" + "="*70)
print(f"📊 STEP 5: Creating train/test split ({TRAIN_SPLIT*100}%/{(1-TRAIN_SPLIT)*100}%)")
print("="*70)

# Shuffle dataset
df_clean = df_clean.sample(frac=1, random_state=42).reset_index(drop=True)

# Split
split_idx = int(len(df_clean) * TRAIN_SPLIT)
df_train = df_clean.iloc[:split_idx].copy()
df_test = df_clean.iloc[split_idx:].copy()

print(f"✅ Split complete!")
print(f"   Training set: {len(df_train)} records ({len(df_train)/len(df_clean)*100:.1f}%)")
print(f"   Test set:     {len(df_test)} records ({len(df_test)/len(df_clean)*100:.1f}%)")

# Save test set for later evaluation
df_test.to_csv('tla_test_set.csv', index=False)
print(f"\n💾 Test set saved to: tla_test_set.csv")


📊 STEP 5: Creating train/test split (99.0%/1.0000000000000009%)
✅ Split complete!
   Training set: 8997 records (99.0%)
   Test set:     91 records (1.0%)

💾 Test set saved to: tla_test_set.csv


In [52]:
## part 7: Generate Embedding

from sentence_transformers import SentenceTransformer

print("\n📥 Loading embedding model...")
# Load model (do this ONCE before the loop)
embedding_model = SentenceTransformer('BAAI/bge-m3')
print(f"✅ Model loaded: BAAI/bge-m3")

def get_embedding_fast(text):
    """Generate embedding using sentence-transformers"""
    try:
        # Generate embedding
        embedding = embedding_model.encode(text, normalize_embeddings=True)
        return embedding.tolist()
    except Exception as e:
        print(f"❌ Error: {e}")
        return np.random.randn(VECTOR_DIM).tolist()

print("\n" + "="*70)
print(f"🔢 STEP 6: Generating embeddings for {len(df_train)} records")
print("="*70)

print(f"\n⚙️ Using model: BAAI/bge-m3")
print(f"   Vector dimension: {VECTOR_DIM}")

# Generate embeddings in batches (MUCH faster!)
batch_size = 32
all_embeddings = []

for i in tqdm(range(0, len(df_train), batch_size), desc="Generating embeddings"):
    batch_end = min(i + batch_size, len(df_train))
    batch_texts = df_train.iloc[i:batch_end]['transliteration_normalized'].tolist()

    try:
        # Process entire batch at once (FAST!)
        batch_embeddings = embedding_model.encode(
            batch_texts,
            normalize_embeddings=True,
            show_progress_bar=False
        )
        all_embeddings.extend(batch_embeddings.tolist())
    except Exception as e:
        print(f"❌ Batch error at {i}: {e}")
        # Fallback: process individually
        for text in batch_texts:
            all_embeddings.append(get_embedding_fast(text))

df_train['embedding'] = all_embeddings

print(f"\n✅ Embedding generation complete!")
print(f"   Total: {len(all_embeddings)} embeddings")
print(f"   Dimension: {len(all_embeddings[0])}")

# Verify
sample_embedding = all_embeddings[0]
print(f"\n📊 Sample embedding (first 10 values):")
print(f"   {sample_embedding[:10]}")


📥 Loading embedding model...
✅ Model loaded: BAAI/bge-m3

🔢 STEP 6: Generating embeddings for 8997 records

⚙️ Using model: BAAI/bge-m3
   Vector dimension: 1024


Generating embeddings:   0%|          | 0/282 [00:00<?, ?it/s]


✅ Embedding generation complete!
   Total: 8997 embeddings
   Dimension: 1024

📊 Sample embedding (first 10 values):
   [0.009475680999457836, 0.012296928092837334, -0.03066054731607437, 0.0029091022443026304, -0.038571588695049286, -0.0011071120388805866, -0.002732239430770278, -0.013377217575907707, 0.02956884168088436, -0.00481629790738225]


In [53]:
## part 8: Exract Lemmas from Lemmettizaton

def extract_lemmas(lemmatization_text):
    """Extract lemma words from lemmatization field"""
    if not isinstance(lemmatization_text, str):
        return []

    lemmas = []
    parts = lemmatization_text.split()

    for part in parts:
        if '|' in part:
            lemma_id, lemma_word = part.split('|', 1)
            # Skip suffixes/particles
            if not lemma_word.startswith('='):
                lemmas.append(lemma_word)

    return lemmas

print("\n" + "="*70)
print("📝 STEP 7: Extracting lemmas")
print("="*70)

df_train['lemmas'] = df_train['lemmatization'].apply(extract_lemmas)

print(f"✅ Lemma extraction complete!")
print(f"\n📋 Sample lemmas:")
for i in range(min(3, len(df_train))):
    lemmas = df_train.iloc[i]['lemmas']
    print(f"   {i+1}. {lemmas[:5]}")


📝 STEP 7: Extracting lemmas
✅ Lemma extraction complete!

📋 Sample lemmas:
   1. ['ḥm-nṯr-Ḫwi̯=f-wꞽ', 'ḥr.ꞽ-sštꜣ']
   2. ['zꜣ', 'sms.w', 'ꞽm.ꞽ-rʾ-zẖꜣ.ww-ꜥ-n-nswt', 'Sšm-nfr']
   3. ['zbi̯', 'ṯw', 'm', 'ꜥḥꜥ.w', 'nfr']


In [54]:
## part 9: Setup Qdrant Vector Database
print("\n" + "="*70)
print("Setting up Qdrant database")
print("="*70)

# Initialize Qdrant (in-memory for development)
# For production, use: QdrantClient(host="localhost", port=6333)
qdrant = QdrantClient(path="qdrant_db")

print(f"✅ Qdrant client initialized (in-memory)")

# Create collection
collection_name = "egyptian_transliterations"

qdrant.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=VECTOR_DIM,
        distance=Distance.COSINE
    )
)

print(f"✅ Collection created: {collection_name}")
print(f"   Vector size: {VECTOR_DIM}")
print(f"   Distance metric: COSINE")


Setting up Qdrant database
✅ Qdrant client initialized (in-memory)
✅ Collection created: egyptian_transliterations
   Vector size: 1024
   Distance metric: COSINE


In [56]:
## part 10: upload data to qdrant


print("\n" + "="*70)
print(f" Uploading {len(df_train)} records to Qdrant")
print("="*70)

# Prepare points
points = []

for idx, row in tqdm(df_train.iterrows(), total=len(df_train), desc="Preparing points"):
    point = PointStruct(
        id=idx,
        vector=row['embedding'],
        payload={
            "transliteration_original": row['transliteration'],
            "transliteration_normalized": row['transliteration_normalized'],
            "lemmas": row['lemmas'],
            "UPOS": row.get('UPOS', ''),
            "glossing": row.get('glossing', ''),
            "translation_de": row['translation']
        }
    )
    points.append(point)

# Upload in batches
batch_size = 100
print(f"\n📦 Uploading in batches of {batch_size}...")

for i in tqdm(range(0, len(points), batch_size), desc="Uploading batches"):
    batch = points[i:i+batch_size]
    qdrant.upsert(
        collection_name=collection_name,
        points=batch
    )

print(f"\n✅ Upload complete!")
print(f"   Total records in database: {len(points)}")





 Uploading 8997 records to Qdrant


Preparing points:   0%|          | 0/8997 [00:00<?, ?it/s]


📦 Uploading in batches of 100...


Uploading batches:   0%|          | 0/90 [00:00<?, ?it/s]


✅ Upload complete!
   Total records in database: 8997


In [57]:
## part 11 : verify Database

print("\n" + "="*70)
print("✅ STEP 10: Verifying database")
print("="*70)

count_info = qdrant.count(
    collection_name=collection_name,
    exact=True
)

print(f"📊 Collection statistics:")
print(f"   Name: {collection_name}")
print(f"   Points count: {count_info.count}")

# Test search
print(f"\n🔍 Testing search functionality...")

test_query = df_train.iloc[0]['transliteration_normalized']
test_embedding = df_train.iloc[0]['embedding']

search_results = qdrant.query_points(
    collection_name=collection_name,
    query=test_embedding,
    limit=3
).points

print(f"\n📝 Test query: {test_query}")
print(f"\n🎯 Top 3 search results:")

for i, result in enumerate(search_results, 1):
    print(f"\n   {i}. Score: {result.score:.4f}")
    print(f"      Transliteration: {result.payload['transliteration_normalized']}")
    print(f"      Translation: {result.payload['translation_de'][:60]}...")



✅ STEP 10: Verifying database
📊 Collection statistics:
   Name: egyptian_transliterations
   Points count: 8997

🔍 Testing search functionality...

📝 Test query: hm-ntjr-khwi=f-wi hr.i-sshta

🎯 Top 3 search results:

   1. Score: 1.0000
      Transliteration: hm-ntjr-khwi=f-wi hr.i-sshta
      Translation: Priester des Cheops und Hüter des Geheimnisses....

   2. Score: 0.9444
      Transliteration: hm-ntjr-khwi=f-wi hr.i-sshta ka=i-n.i-nswt
      Translation: Priester des Cheops und Hüter des Geheimnisses Kai-ni-nisut....

   3. Score: 0.8680
      Transliteration: wꜥb-nswt hr.i-sshta im.i-s.t-ka=i
      Translation: Der Wab-Priester des Königs und Hüter des Geheimnisses Imi-s...


# ═══════════════════════════════════════════════════════════════════
# 🔮 PART 2: RAG Translation Pipeline
# ═══════════════════════════════════════════════════════════════════

## 📦 Part 12: Install Additional Libraries

In [97]:
import os
from google.colab import userdata

# Load Ollama API Key securely from Colab Secrets
OLLAMA_API_KEY = userdata.get('OLLAMA_API_KEY')

if OLLAMA_API_KEY is None:
    raise ValueError("❌ OLLAMA_API_KEY not found in Colab Secrets")

# Set env var for libraries that expect it
os.environ['OLLAMA_API_KEY'] = OLLAMA_API_KEY

# Configuration
LLM_MODEL = "qwen3-vl:235b-instruct-cloud" #"gpt-oss:120b-cloud" #"qwen3-next:80b-cloud" #"qwen3-vl:235b-cloud"
TOP_K_RESULTS = 20

print("🔧 RAG Pipeline Configuration:")
print(f"   LLM Model: {LLM_MODEL}")
print(f"   Top-K Results: {TOP_K_RESULTS}")
print(f"   API Key: ✅ Loaded securely from Colab Secrets")


🔧 RAG Pipeline Configuration:
   LLM Model: qwen3-vl:235b-instruct-cloud
   Top-K Results: 20
   API Key: ✅ Loaded securely from Colab Secrets


In [98]:
!pip install rank-bm25

In [99]:
## part 14: prepare BM25 index for Sparce Search

from rank_bm25 import BM25Okapi

print("\n" + "="*70)
print(" Building BM25 index for sparse search")
print("="*70)

# Tokenize corpus for BM25
corpus_texts = df_train['transliteration_normalized'].tolist()
tokenized_corpus = [text.split() for text in corpus_texts]

# Build BM25 index
bm25 = BM25Okapi(tokenized_corpus)

print(f"✅ BM25 index built!")
print(f"   Documents indexed: {len(tokenized_corpus)}")


 Building BM25 index for sparse search
✅ BM25 index built!
   Documents indexed: 8997


In [100]:
## part 15: Hybrid Search Function
def hybrid_search(query_text, query_embedding, top_k=10, alpha=0.5):
    """
    Perform hybrid search: Dense (Vector) + Sparse (BM25)

    Args:
        query_text: Normalized transliteration query
        query_embedding: Embedding vector of query
        top_k: Number of results to return
        alpha: Weight for dense search (1-alpha for sparse)

    Returns:
        List of search results with scores
    """

    # 1. Dense Search (Vector Similarity)
    dense_results = qdrant.query_points(
        collection_name=collection_name,
        query=query_embedding,
        limit=top_k * 2
    ).points


    # 2. Sparse Search (BM25)
    query_tokens = query_text.split()
    bm25_scores = bm25.get_scores(query_tokens)

    # Get top BM25 indices
    top_bm25_indices = np.argsort(bm25_scores)[-top_k*2:][::-1]

    # 3. Reciprocal Rank Fusion (RRF)
    combined_scores = {}

    # Add dense scores
    for rank, result in enumerate(dense_results):
        doc_id = result.id
        rrf_score = 1 / (rank + 60)  # RRF formula
        combined_scores[doc_id] = {
            'rrf_score': rrf_score,
            'dense_score': result.score,
            'sparse_score': 0,
            'payload': result.payload
        }

    # Add sparse scores
    for rank, idx in enumerate(top_bm25_indices):
        if idx in combined_scores:
            combined_scores[idx]['rrf_score'] += 1 / (rank + 60)
            combined_scores[idx]['sparse_score'] = bm25_scores[idx]
        else:
            # Retrieve payload from Qdrant
            point = qdrant.retrieve(
                collection_name=collection_name,
                ids=[int(idx)]
            )
            if point:
                combined_scores[idx] = {
                    'rrf_score': 1 / (rank + 60),
                    'dense_score': 0,
                    'sparse_score': bm25_scores[idx],
                    'payload': point[0].payload
                }

    # 4. Sort by combined RRF score
    sorted_results = sorted(
        combined_scores.items(),
        key=lambda x: x[1]['rrf_score'],
        reverse=True
    )[:top_k]

    # 5. Format results
    final_results = []
    for doc_id, scores in sorted_results:
        final_results.append({
            'id': doc_id,
            'rrf_score': scores['rrf_score'],
            'dense_score': scores['dense_score'],
            'sparse_score': scores['sparse_score'],
            'payload': scores['payload']
        })

    return final_results

print("✅ Hybrid search function ready!")


✅ Hybrid search function ready!


In [101]:
## part 16: LLM Translation Function
import requests
OLLAMA_API_URL = "https://ollama.com/api/chat"

def translate_with_llm(query_original, query_normalized, retrieved_examples):
    """
    Use LLM to translate Egyptian to German based on retrieved examples
    """

    # Build examples context (same as before)
    examples_text = ""
    for i, example in enumerate(retrieved_examples, 1):
        payload = example['payload']
        examples_text += f"""
Example {i}:
- Original: {payload['transliteration_original']}
- Normalized: {payload['transliteration_normalized']}
- Lemmas: {', '.join(payload['lemmas'][:5]) if payload['lemmas'] else 'N/A'}
- POS Tags: {payload['UPOS']}
- Glossing: {payload['glossing']}
- German: {payload['translation_de']}
---
"""

    # Build prompt (same as before)
    prompt = f"""You are a senior linguist specializing in Earlier Egyptian (Old Egyptian & Early Middle Egyptian),
with strong expertise in morphology, syntax, and historical semantics.

Your task is to translate an Earlier Egyptian transliteration into German
using retrieved linguistic examples ONLY as structural and semantic guidance.

=====================================
QUERY TO TRANSLATE
=====================================

Normalized Transliteration:
{query_normalized}

=====================================
RETRIEVED DATABASE EXAMPLES
=====================================
{examples_text}

=====================================
INSTRUCTIONS
=====================================
Follow these steps carefully:

1. Linguistic Analysis
   - Identify the grammatical category of each word (verb, noun, particle, suffix, etc.)
   - Detect verb tense/aspect, suffix pronouns, and syntactic order (VSO, SVO, nominal clause).

2. Morphological Alignment
   - Compare suffixes, verb forms, and particles with the retrieved examples.
   - Use lemma meanings as semantic hints, not literal translations.

3. Translation Construction
   - Produce a fluent and historically plausible German translation.
   - Adapt word order to correct German syntax.
   - Prefer linguistically conservative interpretations over speculative ones.

4. Uncertainty Handling
   - If multiple readings are possible, choose the most likely one.
   - Briefly mention ambiguity only if it materially affects meaning.

=====================================
STRICT RULES
=====================================
- DO NOT copy any German translation from the examples.
- DO NOT mention example numbers or quote them.
- DO NOT add explanations unless uncertainty exists.
- DO NOT hallucinate missing words.
- Base your output strictly on Earlier Egyptian grammar.

=====================================
OUTPUT FORMAT (STRICT)
=====================================
German Translation: <one clear German sentence>
Confidence: High | Medium | Low
Notes: <only if confidence is Medium or Low>

"""

    # Call Ollama Cloud API with CORRECT endpoint
    try:
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {OLLAMA_API_KEY}"
        }

        # Use Ollama's native format (not OpenAI format)
        payload = {
            "model": LLM_MODEL,
            "messages": [
                {
                    "role": "system",
                    "content": "You are an expert Ancient Egyptian linguist specializing in translating Earlier Egyptian to German."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "stream": False
        }

        response = requests.post(
            OLLAMA_API_URL,
            headers=headers,
            json=payload,
            timeout=240
        )

        if response.status_code == 200:
            result = response.json()
            # Ollama format uses 'message' -> 'content'
            llm_output = result['message']['content']

            # Extract German translation
            import re
            match = re.search(r'German Translation:\s*(.+?)(?:\n|$)', llm_output, re.IGNORECASE)
            if match:
                german_translation = match.group(1).strip()
                return german_translation, llm_output
            else:
                return llm_output.split('\n')[0].strip(), llm_output
        else:
            print(f"❌ API Error: {response.status_code}")
            print(f"   Response: {response.text}")
            return None, None

    except Exception as e:
        print(f"❌ LLM Error: {e}")
        return None, None


In [102]:
## part 17: German to English Tranlslation
from transformers import MarianMTModel, MarianTokenizer

print("\n" + "="*70)
print("Loading German→English translation model")
print("="*70)

# Load MarianMT model
print("📥 Loading MarianMT model...")
de_en_model_name = "Helsinki-NLP/opus-mt-de-en"
de_en_tokenizer = MarianTokenizer.from_pretrained(de_en_model_name)
de_en_model = MarianMTModel.from_pretrained(de_en_model_name)

print(f"✅ Model loaded: {de_en_model_name}")

def translate_german_to_english(german_text):
    """Translate German to English using MarianMT"""
    try:
        # Tokenize
        inputs = de_en_tokenizer(
            german_text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        )

        # Generate translation
        outputs = de_en_model.generate(**inputs)

        # Decode
        english_text = de_en_tokenizer.decode(
            outputs[0],
            skip_special_tokens=True
        )

        return english_text

    except Exception as e:
        print(f"❌ Translation error: {e}")
        return None

print("✅ German→English translation ready!")




Loading German→English translation model
📥 Loading MarianMT model...
✅ Model loaded: Helsinki-NLP/opus-mt-de-en
✅ German→English translation ready!


In [103]:
## part 18: Complete Translation pipeline

def translate_egyptian_to_english(query_original, show_details=True):
    """
    Complete pipeline: Egyptian → German → English

    Args:
        query_original: Original Egyptian transliteration
        show_details: Print intermediate steps

    Returns:
        dict with results
    """

    if show_details:
        print("\n" + "="*70)
        print(f"📝 TRANSLATING: {query_original}")
        print("="*70)

    # Step 1: Normalize query
    query_normalized = normalize_transliteration(query_original)

    if show_details:
        print(f"\n1️⃣ Normalization:")
        print(f"   Original:   {query_original}")
        print(f"   Normalized: {query_normalized}")

    # Step 2: Generate embedding
    query_embedding = embedding_model.encode(
        query_normalized,
        normalize_embeddings=True
    ).tolist()

    if show_details:
        print(f"\n2️⃣ Embedding generated (dim={len(query_embedding)})")

    # Step 3: Hybrid search
    if show_details:
        print(f"\n3️⃣ Hybrid search (Dense + BM25)...")

    search_results = hybrid_search(
        query_text=query_normalized,
        query_embedding=query_embedding,
        top_k=TOP_K_RESULTS
    )

    if show_details:
        print(f"   ✅ Found {len(search_results)} results")
        print(f"\n   📊 Top 3 matches:")
        for i, result in enumerate(search_results[:3], 1):
            print(f"\n   {i}. RRF Score: {result['rrf_score']:.4f}")
            print(f"      Transliteration: {result['payload']['transliteration_normalized']}")
            print(f"      German: {result['payload']['translation_de'][:50]}...")

    # Step 4: LLM Translation (German)
    if show_details:
        print(f"\n4️⃣ LLM Translation (Egyptian → German)...")

    german_translation, llm_full_output = translate_with_llm(
        query_original=query_original,
        query_normalized=query_normalized,
        retrieved_examples=search_results
    )

    if not german_translation:
        return {
            'success': False,
            'error': 'LLM translation failed'
        }

    if show_details:
        print(f"   🇩🇪 German: {german_translation}")

    # Step 5: German → English
    if show_details:
        print(f"\n5️⃣ Translation (German → English)...")

    english_translation = translate_german_to_english(german_translation)

    if not english_translation:
        return {
            'success': False,
            'error': 'German→English translation failed'
        }

    # Final result
    if show_details:
        print("\n" + "="*70)
        print("✅ TRANSLATION COMPLETE")
        print("="*70)
        print(f"🏛️ Egyptian:  {query_original}")
        print(f"🔤 Normalized: {query_normalized}")
        print(f"🇩🇪 German:    {german_translation}")
        print(f"🇬🇧 English:   {english_translation}")
        print("="*70 + "\n")

    return {
        'success': True,
        'query_original': query_original,
        'query_normalized': query_normalized,
        'german': german_translation,
        'english': english_translation,
        'llm_output': llm_full_output,
        'top_matches': search_results[:3]
    }

print("✅ Complete translation pipeline ready!")


✅ Complete translation pipeline ready!


# ========No======================

In [73]:
## part 19: Batch Processing Test set
print("\n" + "="*70)
print("📊 STEP 13: Processing test set")
print("="*70)

# Load test set
print("📥 Loading test set...")
df_test = pd.read_csv('tla_test_set.csv')
print(f"✅ Loaded {len(df_test)} test records")

# Process subset (first 10 for demo)
print(f"\n🔄 Translating first 10 test queries...")
print("(Processing all {len(df_test)} would take ~{len(df_test)*2/60:.1f} minutes)\n")

results = []

for idx in tqdm(range(min(10, len(df_test))), desc="Translating"):
    query = df_test.iloc[idx]['transliteration']

    result = translate_egyptian_to_english(
        query_original=query,
        show_details=False
    )

    if result['success']:
        results.append({
            'query_original': result['query_original'],
            'query_normalized': result['query_normalized'],
            'reference_german': df_test.iloc[idx]['translation'],
            'predicted_german': result['german'],
            'predicted_english': result['english']
        })

# Create results dataframe
df_results = pd.DataFrame(results)

print(f"\n✅ Processed {len(results)} queries successfully!")



📊 STEP 13: Processing test set
📥 Loading test set...
✅ Loaded 91 test records

🔄 Translating first 10 test queries...
(Processing all {len(df_test)} would take ~{len(df_test)*2/60:.1f} minutes)



Translating:   0%|          | 0/10 [00:00<?, ?it/s]


✅ Processed 10 queries successfully!


## part 20: Display result

In [74]:
print("\n" + "="*70)
print("📈 SAMPLE RESULTS")
print("="*70)

for i in range(min(5, len(df_results))):
    row = df_results.iloc[i]
    print(f"\n{'─'*70}")
    print(f"Query {i+1}:")
    print(f"{'─'*70}")
    print(f"🏛️ Egyptian:    {row['query_original']}")
    print(f"🔤 Normalized:  {row['query_normalized']}")
    print(f"📖 Reference:   {row['reference_german']}")
    print(f"🤖 Predicted:   {row['predicted_german']}")
    print(f"🇬🇧 English:    {row['predicted_english']}")


📈 SAMPLE RESULTS

──────────────────────────────────────────────────────────────────────
Query 1:
──────────────────────────────────────────────────────────────────────
🏛️ Egyptian:    smr-wꜥ.tꞽ ẖr(.ꞽ)-ḥ(ꜣ)b(.t) (ꞽ)m(.ꞽ)-r(ʾ)-(ꞽ)ꜥw ḥr(.w)-ḫwi̯=f
🔤 Normalized:  smr-wꜥ.ti khr.i-hab.t im.i-ra-iꜥw hr.w-khwi
📖 Reference:   Der Einzige Freund, der Vorlesepriester, der Vorsteher der fremdsprachigen Truppe, Harchuf.
🤖 Predicted:   Der einzige Freund des Königs, der Vorlesepriester, der Vorsteher der fremdsprachigen Truppe, Harchuf.
🇬🇧 English:    The King's only friend, the presiding priest, the ruler of the foreign language troupe, Harchuf.

──────────────────────────────────────────────────────────────────────
Query 2:
──────────────────────────────────────────────────────────────────────
🏛️ Egyptian:    ḫꜣ ꜣpd ḫꜣ ꜣpd ḫꜣ mnḫ.t ḫꜣ ꞽḥ ḫꜣ tʾ ḥnq.t pꜣ.t ḫꜣ ꜣpd ḫꜣ ꜣpd ḫꜣ šs
🔤 Normalized:  kha apd kha apd kha mnkh.t kha ih kha ta hnq.t pa.t kha apd kha apd kha shs
📖 Reference:   Tausend an Geflüg

# =============NO=============================

# ═══════════════════════════════════════════════════════════════════
# 📊 EVALUATION METRICS FOR EGYPTIAN TRANSLITERATION RAG SYSTEM
# ═══════════════════════════════════════════════════════════════════

## 📦 Part 24: Install Evaluation Libraries

In [104]:
print(len(df_test))
print(df_test.columns.tolist())
df_test.head()

25
['transliteration', 'lemmatization', 'UPOS', 'glossing', 'translation', 'transliteration_normalized']


,transliteration,lemmatization,UPOS,glossing,translation,transliteration_normalized
0,smr-wꜥ.tꞽ ẖr(.ꞽ)-ḥ(ꜣ)b(.t) (ꞽ)m(.ꞽ)-r(ʾ)-(ꞽ)ꜥw...,400142|smr-wꜥ.tꞽ 124340|ẖr.ꞽ-ḥꜣb.t 400011|ꞽm.ꞽ...,NOUN NOUN NOUN PROPN,TITL TITL TITL PERSN,"Der Einzige Freund, der Vorlesepriester, der V...",smr-wꜥ.ti khr.i-hab.t im.i-ra-iꜥw hr.w-khwi
1,ḫꜣ ꜣpd ḫꜣ ꜣpd ḫꜣ mnḫ.t ḫꜣ ꞽḥ ḫꜣ tʾ ḥnq.t pꜣ.t ...,113110|ḫꜣ 107|ꜣpd 113110|ḫꜣ 107|ꜣpd 113110|ḫꜣ ...,NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN NOUN N...,N.m N.m N.m N.m N.m N.f N.m N.m N.m N.m N.f N....,"Tausend an Geflügel, tausend an Geflügel, taus...",kha apd kha apd kha mnkh.t kha ih kha ta hnq.t...
2,ꞽ:nḏ ḥr =k mnw m pr.t.PL =f qꜣ šw.tꞽ zꜣ wsꞽr m...,91190|nḏ+(ḥr) 107510|ḥr 10110|=k 70530|Mnw 643...,VERB NOUN PRON PROPN ADP NOUN PRON ADJ NOUN NO...,V N.m:stpr -2sg.m DIVN PREP N.f:pl:stpr -3sg.m...,"Sei gegrüßt, Min bei seinen Prozessionen, mit ...",i:ndj hr mnw m pr.t.pl qa shw.ti za wsir msi.n...
3,šdi̯.t kꜣ rḫ.yt,158710|šdi̯ 162890|kꜣ 95820|rḫ.yt,VERB NOUN NOUN,V\inf N.m N.f,Darbringen der Speisen (für die) Untertanen,shdi.t ka rkh.yt
4,(w)sr(.w) n(ꞽ).t m n =k ꞽr(.t)-ḥr.w šmi̯.t =f ...,49461|Wsꞽr 702960|Nꞽ.t 67780|mꞽ 400055|n 10110...,NOUN PROPN VERB ADP PRON NOUN VERB PRON ADP PRON,TITL PERSN V\imp.sg PREP:stpr -2sg.m N.f V\rel...,"Osiris Neith, nimm dir das Horusauge, zu dem e...",wsr.w ni.t m n ir.t-hr.w shmi.t ir


In [105]:
## 📦 Part 19: Install Evaluation Libraries
import sys
print("\n📦 Installing all evaluation libraries...")
eval_packages = [
    'nltk',
    'rouge-score',
    'sacrebleu',
]

for package in eval_packages:
    subprocess.run([sys.executable, '-m', 'pip', 'install', package, '--break-system-packages', '-q'])

import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
from sacrebleu.metrics import CHRF
import warnings
warnings.filterwarnings('ignore')

# Download NLTK data
print("📥 Downloading NLTK data...")
try:
    nltk.download('wordnet', quiet=True)
    nltk.download('punkt', quiet=True)
    nltk.download('omw-1.4', quiet=True)
    print("✅ NLTK data ready!")
except:
    print("⚠️ NLTK download warning (may still work)")


📦 Installing all evaluation libraries...
📥 Downloading NLTK data...
✅ NLTK data ready!


In [106]:
## 📊 Part 20: Define Evaluation Metrics

# ============================================================================
# TRANSLATION QUALITY METRICS
# ============================================================================

def calculate_bleu(reference, hypothesis):
    """Calculate BLEU score (0-100)"""
    try:
        reference_tokens = reference.lower().split()
        hypothesis_tokens = hypothesis.lower().split()
        smoothing = SmoothingFunction()
        bleu_score = sentence_bleu(
            [reference_tokens],
            hypothesis_tokens,
            smoothing_function=smoothing.method1
        )
        return bleu_score * 100
    except:
        return 0.0

def calculate_rouge(reference, hypothesis):
    """Calculate ROUGE scores"""
    try:
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        scores = scorer.score(reference, hypothesis)
        return {
            'rouge1': scores['rouge1'].fmeasure * 100,
            'rouge2': scores['rouge2'].fmeasure * 100,
            'rougeL': scores['rougeL'].fmeasure * 100
        }
    except:
        return {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0}

def calculate_meteor(reference, hypothesis):
    """Calculate METEOR score (0-100)"""
    try:
        reference_tokens = reference.lower().split()
        hypothesis_tokens = hypothesis.lower().split()
        meteor = meteor_score([reference_tokens], hypothesis_tokens)
        return meteor * 100
    except:
        return 0.0

def calculate_chrf(reference, hypothesis):
    """Calculate chrF score (0-100)"""
    try:
        chrf = CHRF()
        score = chrf.sentence_score(hypothesis, [reference])
        return score.score  # Already 0-100
    except:
        return 0.0

def calculate_exact_match(reference, hypothesis):
    """Calculate exact match"""
    return 100.0 if reference.strip().lower() == hypothesis.strip().lower() else 0.0

def calculate_word_overlap(reference, hypothesis):
    """Calculate word-level overlap percentage"""
    try:
        ref_words = set(reference.lower().split())
        hyp_words = set(hypothesis.lower().split())
        if len(ref_words) == 0:
            return 0.0
        overlap = len(ref_words.intersection(hyp_words))
        return (overlap / len(ref_words)) * 100
    except:
        return 0.0

# ============================================================================
# RETRIEVAL QUALITY METRICS
# ============================================================================

def calculate_recall_at_k(reference_german, retrieved_examples, k_values=[1, 3, 5, 10]):
    """
    Calculate Recall@K - checks if reference appears in top K results

    Args:
        reference_german: The ground truth German translation
        retrieved_examples: List of retrieved examples from RAG
        k_values: List of K values to calculate recall for

    Returns:
        Dict with Recall@K for each K
    """
    recalls = {}

    for k in k_values:
        found = False
        for i, example in enumerate(retrieved_examples[:k]):
            if i >= len(retrieved_examples):
                break
            retrieved_german = example['payload']['translation_de']
            # Check if the reference matches (exact or high similarity)
            if reference_german.strip().lower() == retrieved_german.strip().lower():
                found = True
                break

        recalls[f'recall@{k}'] = 100.0 if found else 0.0

    return recalls

def calculate_mrr(reference_german, retrieved_examples):
    """
    Calculate Mean Reciprocal Rank (MRR)

    MRR = 1 / rank of first relevant result
    If no relevant result found, MRR = 0

    Args:
        reference_german: The ground truth German translation
        retrieved_examples: List of retrieved examples from RAG

    Returns:
        MRR score (0-100)
    """
    for i, example in enumerate(retrieved_examples):
        retrieved_german = example['payload']['translation_de']
        # Check if this is a relevant result
        if reference_german.strip().lower() == retrieved_german.strip().lower():
            # Rank starts at 1, not 0
            mrr = 1.0 / (i + 1)
            return mrr * 100  # Convert to percentage

    # No relevant result found
    return 0.0

def calculate_average_retrieval_score(retrieved_examples, top_k=10):
    """
    Calculate average retrieval score from top K results

    Args:
        retrieved_examples: List of retrieved examples with scores
        top_k: Number of top results to consider

    Returns:
        Average RRF score (0-100)
    """
    if not retrieved_examples:
        return 0.0

    scores = [example['rrf_score'] for example in retrieved_examples[:top_k]]
    avg_score = np.mean(scores) if scores else 0.0
    return avg_score * 100  # Convert to percentage

print("✅ All evaluation metrics defined!")


✅ All evaluation metrics defined!


In [107]:
## 🔄 Part 21: Process RAG Test Set with ALL Metrics

print("\n" + "="*70)
print("🚀 PROCESSING RAG TEST SET WITH ALL METRICS")
print("="*70)

# Load test set
print("\n📥 Loading test set...")
df_test = pd.read_csv('tla_test_set.csv')[:25]  # Limit for demo
print(f"✅ Loaded {len(df_test)} test records")

# Initialize results storage
test_results = []
failed_translations = []

print(f"\n🔄 Processing {len(df_test)} test samples...")
print("⏱️ Estimated time: ~{:.1f} minutes\n".format(len(df_test) * 3 / 60))

# Process each test sample
for idx in tqdm(range(len(df_test)), desc="RAG Translation"):
    try:
        # Get query
        query_original = df_test.iloc[idx]['transliteration']
        reference_german = df_test.iloc[idx]['translation']

        # Translate Egyptian → German → English using RAG
        result = translate_egyptian_to_english(
            query_original=query_original,
            show_details=False
        )

        if result['success']:
            # Translate reference German → English
            reference_english = translate_german_to_english(reference_german)

            if reference_english:
                # Store results (including retrieval info)
                test_results.append({
                    'sample_id': idx,
                    'transliteration': query_original,
                    'transliteration_normalized': result['query_normalized'],
                    'reference_german': reference_german,
                    'reference_english': reference_english,
                    'predicted_german': result['german'],
                    'predicted_english': result['english'],
                    'top_matches': result['top_matches']  # Store retrieval results
                })
            else:
                failed_translations.append({
                    'sample_id': idx,
                    'reason': 'Reference translation to English failed'
                })
        else:
            failed_translations.append({
                'sample_id': idx,
                'reason': result.get('error', 'RAG translation failed')
            })

    except Exception as e:
        failed_translations.append({
            'sample_id': idx,
            'reason': f'Exception: {str(e)}'
        })
        continue

# Create results DataFrame
df_test_results = pd.DataFrame(test_results)

print(f"\n✅ RAG Processing complete!")
print(f"   Successful: {len(test_results)}")
print(f"   Failed: {len(failed_translations)}")


🚀 PROCESSING RAG TEST SET WITH ALL METRICS

📥 Loading test set...
✅ Loaded 25 test records

🔄 Processing 25 test samples...
⏱️ Estimated time: ~1.2 minutes



RAG Translation:   0%|          | 0/25 [00:00<?, ?it/s]


✅ RAG Processing complete!
   Successful: 25
   Failed: 0


In [108]:
## 📊 Part 22: Calculate ALL Metrics (Translation + Retrieval)

print("\n" + "="*70)
print("📊 CALCULATING ALL METRICS FOR RAG SYSTEM")
print("="*70)

metrics_list = []

print(f"\n🔄 Computing all metrics for {len(df_test_results)} translations...\n")

for idx, row in tqdm(df_test_results.iterrows(), total=len(df_test_results), desc="Computing all metrics"):
    reference_english = row['reference_english']
    hypothesis_english = row['predicted_english']
    reference_german = row['reference_german']
    retrieved_examples = row['top_matches']

    # Calculate translation quality metrics
    rouge_scores = calculate_rouge(reference_english, hypothesis_english)

    # Calculate retrieval quality metrics
    recall_scores = calculate_recall_at_k(reference_german, retrieved_examples, k_values=[1, 3, 5, 10, 20])
    mrr_score = calculate_mrr(reference_german, retrieved_examples)
    avg_retrieval_score = calculate_average_retrieval_score(retrieved_examples, top_k=10)

    # Combine all metrics
    metrics = {
        'sample_id': row['sample_id'],
        # Translation Quality Metrics
        'bleu': calculate_bleu(reference_english, hypothesis_english),
        'rouge1': rouge_scores['rouge1'],
        'rouge2': rouge_scores['rouge2'],
        'rougeL': rouge_scores['rougeL'],
        'meteor': calculate_meteor(reference_english, hypothesis_english),
        'chrf': calculate_chrf(reference_english, hypothesis_english),
        'exact_match': calculate_exact_match(reference_english, hypothesis_english),
        'word_overlap': calculate_word_overlap(reference_english, hypothesis_english),
        # Retrieval Quality Metrics
        'recall@1': recall_scores['recall@1'],
        'recall@3': recall_scores['recall@3'],
        'recall@5': recall_scores['recall@5'],
        'recall@10': recall_scores['recall@10'],
        'recall@20': recall_scores['recall@20'],
        'mrr': mrr_score,
        'avg_retrieval_score': avg_retrieval_score
    }

    metrics_list.append(metrics)

# Create metrics DataFrame
df_metrics = pd.DataFrame(metrics_list)

print("✅ All metrics calculation complete!")


📊 CALCULATING ALL METRICS FOR RAG SYSTEM

🔄 Computing all metrics for 25 translations...



Computing all metrics:   0%|          | 0/25 [00:00<?, ?it/s]

✅ All metrics calculation complete!


In [109]:

## 📈 Part 23: Display Complete RAG System Summary

print("\n" + "="*70)
print("📈 COMPLETE RAG SYSTEM EVALUATION SUMMARY")
print("="*70)

# Calculate averages for all metrics
avg_metrics = {
    # Translation Quality
    'BLEU': df_metrics['bleu'].mean(),
    'ROUGE-1': df_metrics['rouge1'].mean(),
    'ROUGE-2': df_metrics['rouge2'].mean(),
    'ROUGE-L': df_metrics['rougeL'].mean(),
    'METEOR': df_metrics['meteor'].mean(),
    'chrF': df_metrics['chrf'].mean(),
    'Exact Match': df_metrics['exact_match'].mean(),
    'Word Overlap': df_metrics['word_overlap'].mean(),
    # Retrieval Quality
    'Recall@1': df_metrics['recall@1'].mean(),
    'Recall@3': df_metrics['recall@3'].mean(),
    'Recall@5': df_metrics['recall@5'].mean(),
    'Recall@10': df_metrics['recall@10'].mean(),
    'Recall@20': df_metrics['recall@20'].mean(),
    'MRR': df_metrics['mrr'].mean(),
    'Avg Retrieval Score': df_metrics['avg_retrieval_score'].mean()
}

# Quality emoji function
def get_quality_emoji(metric_name, score):
    """Get quality emoji based on metric and score"""
    if metric_name in ['Recall@1', 'Recall@3', 'Exact Match']:
        return '🟢' if score > 20 else '🟡' if score > 5 else '🔴'
    elif 'Recall' in metric_name:
        return '🟢' if score > 40 else '🟡' if score > 20 else '🔴'
    elif metric_name == 'MRR':
        return '🟢' if score > 30 else '🟡' if score > 15 else '🔴'
    else:
        return '🟢' if score > 50 else '🟡' if score > 30 else '🔴'

print("\n📊 TRANSLATION QUALITY METRICS:")
print("─" * 70)
translation_metrics = ['BLEU', 'ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'METEOR', 'chrF', 'Exact Match', 'Word Overlap']
for metric in translation_metrics:
    score = avg_metrics[metric]
    emoji = get_quality_emoji(metric, score)
    print(f"   {emoji} {metric:20s}: {score:6.2f}%")

print("\n📊 RETRIEVAL QUALITY METRICS:")
print("─" * 70)
retrieval_metrics = ['Recall@1', 'Recall@3', 'Recall@5', 'Recall@10', 'Recall@20', 'MRR', 'Avg Retrieval Score']
for metric in retrieval_metrics:
    score = avg_metrics[metric]
    emoji = get_quality_emoji(metric, score)
    print(f"   {emoji} {metric:20s}: {score:6.2f}%")

# Distribution statistics for key metrics
print("\n📈 KEY METRIC DISTRIBUTIONS:")
print("─" * 70)

for metric_name, metric_col in [('BLEU', 'bleu'), ('METEOR', 'meteor'), ('Recall@10', 'recall@10'), ('MRR', 'mrr')]:
    scores = df_metrics[metric_col]
    print(f"\n{metric_name}:")
    print(f"   Min:    {scores.min():6.2f}%")
    print(f"   25%:    {scores.quantile(0.25):6.2f}%")
    print(f"   Median: {scores.median():6.2f}%")
    print(f"   75%:    {scores.quantile(0.75):6.2f}%")
    print(f"   Max:    {scores.max():6.2f}%")
    print(f"   Std:    {scores.std():6.2f}%")

print("\n" + "="*70)



📈 COMPLETE RAG SYSTEM EVALUATION SUMMARY

📊 TRANSLATION QUALITY METRICS:
──────────────────────────────────────────────────────────────────────
   🔴 BLEU                :  25.47%
   🟢 ROUGE-1             :  51.22%
   🟡 ROUGE-2             :  35.23%
   🟡 ROUGE-L             :  49.76%
   🟡 METEOR              :  40.05%
   🟡 chrF                :  45.58%
   🔴 Exact Match         :   4.00%
   🟡 Word Overlap        :  40.80%

📊 RETRIEVAL QUALITY METRICS:
──────────────────────────────────────────────────────────────────────
   🔴 Recall@1            :   0.00%
   🔴 Recall@3            :   0.00%
   🔴 Recall@5            :   0.00%
   🔴 Recall@10           :   0.00%
   🔴 Recall@20           :   0.00%
   🔴 MRR                 :   0.00%
   🔴 Avg Retrieval Score :   3.06%

📈 KEY METRIC DISTRIBUTIONS:
──────────────────────────────────────────────────────────────────────

BLEU:
   Min:      0.00%
   25%:      1.89%
   Median:   8.84%
   75%:     50.65%
   Max:    100.00%
   Std:     30.80%

METEOR:

In [110]:
## 📊 Part 24: Visual Comparison Charts

print("\n" + "="*70)
print("📊 VISUAL METRIC COMPARISON")
print("="*70)

# Translation metrics
print("\n🎯 Translation Quality:")
print("─" * 70)
for metric, score in list(avg_metrics.items())[:8]:
    bar_length = int((score / 100) * 50)
    bar = '█' * bar_length
    print(f"{metric:20s} {bar} {score:6.2f}%")

# Retrieval metrics
print("\n🔍 Retrieval Quality:")
print("─" * 70)
for metric, score in list(avg_metrics.items())[8:]:
    bar_length = int((score / 100) * 50)
    bar = '█' * bar_length
    print(f"{metric:20s} {bar} {score:6.2f}%")

print("\n" + "="*70)


📊 VISUAL METRIC COMPARISON

🎯 Translation Quality:
──────────────────────────────────────────────────────────────────────
BLEU                 ████████████  25.47%
ROUGE-1              █████████████████████████  51.22%
ROUGE-2              █████████████████  35.23%
ROUGE-L              ████████████████████████  49.76%
METEOR               ████████████████████  40.05%
chrF                 ██████████████████████  45.58%
Exact Match          ██   4.00%
Word Overlap         ████████████████████  40.80%

🔍 Retrieval Quality:
──────────────────────────────────────────────────────────────────────
Recall@1                0.00%
Recall@3                0.00%
Recall@5                0.00%
Recall@10               0.00%
Recall@20               0.00%
MRR                     0.00%
Avg Retrieval Score  █   3.06%



In [111]:
## 📊 Part 25: Process LLM-Only Test Set

print("\n" + "="*70)
print("🚀 PROCESSING LLM-ONLY TEST SET")
print("="*70)

# LLM-only translation function (simplified)

def translate_with_llm_only(query_original, query_normalized):
    """
    Direct LLM translation WITHOUT RAG retrieval
    Only uses the LLM's knowledge
    """

    # Simple prompt without retrieved examples
    prompt = f"""You are an expert linguist specialized in Earlier Egyptian grammar and historical translation.

Translate the following Earlier Egyptian transliteration into German.
Use a conservative, grammar-based interpretation.
Do not modernize meanings or add implied words.

Egyptian Transliteration:
{query_original}

Output ONLY the German translation.
Do not add explanations, comments, or alternative readings.

German Translation:
"""

    try:
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {OLLAMA_API_KEY}"
        }

        payload = {
            "model": LLM_MODEL,
            "messages": [
                {
                    "role": "system",
                    "content": "You are an expert Ancient Egyptian linguist. Translate Earlier Egyptian to German."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "stream": False
        }

        response = requests.post(
            OLLAMA_API_URL,
            headers=headers,
            json=payload,
            timeout=240
        )

        if response.status_code == 200:
            result = response.json()
            llm_output = result['message']['content']

            # Clean the response
            german_translation = llm_output.strip()
            # Remove "German Translation:" prefix if present
            german_translation = re.sub(r'^German Translation:\s*', '', german_translation, flags=re.IGNORECASE)

            return german_translation.strip()
        else:
            print(f"❌ API Error: {response.status_code}")
            return None

    except Exception as e:
        print(f"❌ LLM Error: {e}")
        return None

print("✅ LLM-only translation function ready!")

# Process LLM-only
llm_only_results = []
llm_only_failed = []

print(f"\n🔄 Processing {len(df_test)} samples with LLM-only...")

for idx in tqdm(range(len(df_test)), desc="LLM-only translation"):
    try:
        query_original = df_test.iloc[idx]['transliteration']
        query_normalized = normalize_transliteration(query_original)
        reference_german = df_test.iloc[idx]['translation']

        german_translation = translate_with_llm_only(query_original, query_normalized)

        if german_translation:
            english_translation = translate_german_to_english(german_translation)

            if english_translation:
                # Get reference English from RAG results
                if idx < len(df_test_results):
                    reference_english = df_test_results.iloc[idx]['reference_english']
                else:
                    reference_english = translate_german_to_english(reference_german)

                llm_only_results.append({
                    'sample_id': idx,
                    'transliteration': query_original,
                    'reference_german': reference_german,
                    'reference_english': reference_english,
                    'predicted_german_llm': german_translation,
                    'predicted_english_llm': english_translation
                })
            else:
                llm_only_failed.append({'sample_id': idx, 'reason': 'English translation failed'})
        else:
            llm_only_failed.append({'sample_id': idx, 'reason': 'LLM translation failed'})

    except Exception as e:
        llm_only_failed.append({'sample_id': idx, 'reason': f'Exception: {str(e)}'})
        continue

df_llm_only = pd.DataFrame(llm_only_results)

print(f"\n✅ LLM-only processing complete!")
print(f"   Successful: {len(llm_only_results)}")
print(f"   Failed: {len(llm_only_failed)}")


🚀 PROCESSING LLM-ONLY TEST SET
✅ LLM-only translation function ready!

🔄 Processing 25 samples with LLM-only...


LLM-only translation:   0%|          | 0/25 [00:00<?, ?it/s]


✅ LLM-only processing complete!
   Successful: 25
   Failed: 0


In [112]:
## 🏆 Part 26: Calculate Metrics for LLM-Only

print("\n" + "="*70)
print("📊 CALCULATING METRICS FOR LLM-ONLY")
print("="*70)

llm_only_metrics = []

print(f"\n🔄 Computing metrics for {len(df_llm_only)} LLM-only translations...\n")

for idx, row in tqdm(df_llm_only.iterrows(), total=len(df_llm_only), desc="Computing LLM-only metrics"):
    reference = row['reference_english']
    hypothesis = row['predicted_english_llm']

    rouge_scores = calculate_rouge(reference, hypothesis)

    # LLM-only has NO retrieval metrics (all 0)
    metrics = {
        'sample_id': row['sample_id'],
        # Translation Quality Metrics
        'bleu': calculate_bleu(reference, hypothesis),
        'rouge1': rouge_scores['rouge1'],
        'rouge2': rouge_scores['rouge2'],
        'rougeL': rouge_scores['rougeL'],
        'meteor': calculate_meteor(reference, hypothesis),
        'chrf': calculate_chrf(reference, hypothesis),
        'exact_match': calculate_exact_match(reference, hypothesis),
        'word_overlap': calculate_word_overlap(reference, hypothesis),
        # Retrieval metrics = 0 (no retrieval)
        'recall@1': 0.0,
        'recall@3': 0.0,
        'recall@5': 0.0,
        'recall@10': 0.0,
        'recall@20': 0.0,
        'mrr': 0.0,
        'avg_retrieval_score': 0.0
    }

    llm_only_metrics.append(metrics)

df_llm_only_metrics = pd.DataFrame(llm_only_metrics)

print("✅ LLM-only metrics calculation complete!")


📊 CALCULATING METRICS FOR LLM-ONLY

🔄 Computing metrics for 25 LLM-only translations...



Computing LLM-only metrics:   0%|          | 0/25 [00:00<?, ?it/s]

✅ LLM-only metrics calculation complete!


In [113]:
##  Part 27: COMPREHENSIVE COMPARISON

print("\n" + "="*70)
print("🆚 COMPREHENSIVE RAG vs LLM-ONLY COMPARISON")
print("="*70)

# Calculate averages for LLM-only
llm_only_averages = {
    'BLEU': df_llm_only_metrics['bleu'].mean(),
    'ROUGE-1': df_llm_only_metrics['rouge1'].mean(),
    'ROUGE-2': df_llm_only_metrics['rouge2'].mean(),
    'ROUGE-L': df_llm_only_metrics['rougeL'].mean(),
    'METEOR': df_llm_only_metrics['meteor'].mean(),
    'chrF': df_llm_only_metrics['chrf'].mean(),
    'Exact Match': df_llm_only_metrics['exact_match'].mean(),
    'Word Overlap': df_llm_only_metrics['word_overlap'].mean(),
    'Recall@1': 0.0,
    'Recall@3': 0.0,
    'Recall@5': 0.0,
    'Recall@10': 0.0,
    'Recall@20': 0.0,
    'MRR': 0.0,
    'Avg Retrieval Score': 0.0
}

print("\n📊 COMPLETE METRICS COMPARISON:")
print("─" * 100)
print(f"{'Metric':<25} {'RAG System':<15} {'LLM-Only':<15} {'Difference':<15} {'Winner':<15}")
print("─" * 100)

for metric in avg_metrics.keys():
    rag_score = avg_metrics[metric]
    llm_score = llm_only_averages[metric]
    diff = rag_score - llm_score

    if 'Recall' in metric or metric == 'MRR' or metric == 'Avg Retrieval Score':
        winner = "🏆 RAG (only)" if diff > 0 else "N/A"
    else:
        winner = "🏆 RAG" if diff > 0 else "🏆 LLM" if diff < 0 else "🤝 Tie"

    print(f"{metric:<25} {rag_score:>6.2f}%        {llm_score:>6.2f}%        {diff:>+6.2f}%       {winner:<15}")

print("─" * 100)



🆚 COMPREHENSIVE RAG vs LLM-ONLY COMPARISON

📊 COMPLETE METRICS COMPARISON:
────────────────────────────────────────────────────────────────────────────────────────────────────
Metric                    RAG System      LLM-Only        Difference      Winner         
────────────────────────────────────────────────────────────────────────────────────────────────────
BLEU                       25.47%          2.55%        +22.92%       🏆 RAG          
ROUGE-1                    51.22%         19.00%        +32.22%       🏆 RAG          
ROUGE-2                    35.23%          5.26%        +29.97%       🏆 RAG          
ROUGE-L                    49.76%         17.25%        +32.51%       🏆 RAG          
METEOR                     40.05%         10.32%        +29.74%       🏆 RAG          
chrF                       45.58%         15.29%        +30.29%       🏆 RAG          
Exact Match                 4.00%          0.00%         +4.00%       🏆 RAG          
Word Overlap               40.

In [114]:
## 📦 Part 29: Detailed Comparison Analysis

print("\n" + "="*70)
print("📈 DETAILED COMPARISON ANALYSIS")
print("="*70)

# Statistical comparison
comparison_stats = []

for metric_col in ['bleu', 'rouge1', 'meteor', 'chrf', 'recall@10', 'mrr']:
    rag_scores = df_metrics[metric_col].values
    llm_scores = df_llm_only_metrics[metric_col].values

    stats = {
        'Metric': metric_col.upper(),
        'RAG_Mean': rag_scores.mean(),
        'RAG_Median': np.median(rag_scores),
        'RAG_Std': rag_scores.std(),
        'LLM_Mean': llm_scores.mean(),
        'LLM_Median': np.median(llm_scores),
        'LLM_Std': llm_scores.std(),
        'Mean_Diff': rag_scores.mean() - llm_scores.mean()
    }

    comparison_stats.append(stats)

df_comparison_stats = pd.DataFrame(comparison_stats)

print("\n📊 Statistical Summary:")
print("─" * 100)
print(df_comparison_stats.to_string(index=False))
print("─" * 100)


📈 DETAILED COMPARISON ANALYSIS

📊 Statistical Summary:
────────────────────────────────────────────────────────────────────────────────────────────────────
   Metric  RAG_Mean  RAG_Median   RAG_Std  LLM_Mean  LLM_Median   LLM_Std  Mean_Diff
     BLEU 25.474183    8.839449 30.175759  2.552323    1.888980  2.665147  22.921861
   ROUGE1 51.224286   50.000000 29.189863 19.004810   17.391304 12.678676  32.219476
   METEOR 40.053982   34.307065 29.262594 10.317260    8.620690  8.129328  29.736722
     CHRF 45.580489   37.711307 29.377861 15.285760   15.007055  6.607843  30.294729
RECALL@10  0.000000    0.000000  0.000000  0.000000    0.000000  0.000000   0.000000
      MRR  0.000000    0.000000  0.000000  0.000000    0.000000  0.000000   0.000000
────────────────────────────────────────────────────────────────────────────────────────────────────


In [115]:
## 📊 Part 30: Win/Loss Analysis

print("\n" + "="*70)
print("🏆 SAMPLE-BY-SAMPLE WIN/LOSS ANALYSIS")
print("="*70)

wins = {'RAG': 0, 'LLM': 0, 'Tie': 0}

for metric in ['bleu', 'meteor', 'chrf']:
    metric_wins = {'RAG': 0, 'LLM': 0, 'Tie': 0}

    for i in range(min(len(df_metrics), len(df_llm_only_metrics))):
        rag_score = df_metrics.iloc[i][metric]
        llm_score = df_llm_only_metrics.iloc[i][metric]

        if rag_score > llm_score:
            metric_wins['RAG'] += 1
            wins['RAG'] += 1
        elif llm_score > rag_score:
            metric_wins['LLM'] += 1
            wins['LLM'] += 1
        else:
            metric_wins['Tie'] += 1
            wins['Tie'] += 1

    total = sum(metric_wins.values())
    print(f"\n{metric.upper()} Wins:")
    print(f"  RAG:      {metric_wins['RAG']:3d} ({metric_wins['RAG']/total*100:5.1f}%)")
    print(f"  LLM-Only: {metric_wins['LLM']:3d} ({metric_wins['LLM']/total*100:5.1f}%)")
    print(f"  Tie:      {metric_wins['Tie']:3d} ({metric_wins['Tie']/total*100:5.1f}%)")

total_comparisons = sum(wins.values())
print(f"\n{'─' * 70}")
print(f"Overall Wins (across all metrics):")
print(f"  🏆 RAG:      {wins['RAG']:3d} ({wins['RAG']/total_comparisons*100:5.1f}%)")
print(f"  🏆 LLM-Only: {wins['LLM']:3d} ({wins['LLM']/total_comparisons*100:5.1f}%)")
print(f"  🤝 Tie:      {wins['Tie']:3d} ({wins['Tie']/total_comparisons*100:5.1f}%)")
print("=" * 70)

## 💾 Part 32: Save All Results

print("\n" + "="*70)
print("💾 SAVING ALL RESULTS")
print("="*70)

# Save RAG comprehensive results
df_test_results_full = df_test_results.copy()
for metric in df_metrics.columns:
    if metric != 'sample_id':
        df_test_results_full[metric] = df_metrics[metric].values

df_test_results_full.to_csv('rag_comprehensive_results.csv', index=False)
print(f"✅ RAG comprehensive results saved to: rag_comprehensive_results.csv")

# Save LLM-only comprehensive results
df_llm_only_full = df_llm_only.copy()
for metric in df_llm_only_metrics.columns:
    if metric != 'sample_id':
        df_llm_only_full[metric] = df_llm_only_metrics[metric].values

df_llm_only_full.to_csv('llm_only_comprehensive_results.csv', index=False)
print(f"✅ LLM-only comprehensive results saved to: llm_only_comprehensive_results.csv")

# Save comparison summary
comparison_summary = pd.DataFrame([
    {'System': 'RAG', **{f'{k}': v for k, v in avg_metrics.items()}},
    {'System': 'LLM-Only', **{f'{k}': v for k, v in llm_only_averages.items()}},
    {'System': 'Difference (RAG - LLM)', **{f'{k}': avg_metrics[k] - llm_only_averages[k] for k in avg_metrics.keys()}}
])

comparison_summary.to_csv('complete_comparison_summary.csv', index=False)
print(f"✅ Comparison summary saved to: complete_comparison_summary.csv")

# Save statistical comparison
df_comparison_stats.to_csv('comparison_statistics.csv', index=False)
print(f"✅ Statistics saved to: comparison_statistics.csv")

print("\n" + "="*70)

## ✅ Part 33: FINAL COMPREHENSIVE REPORT

print("\n" + "="*70)
print("✅ FINAL COMPREHENSIVE EVALUATION REPORT")
print("="*70)

# Determine overall winner
rag_total = sum(avg_metrics.values())
llm_total = sum(llm_only_averages.values())
overall_winner = "RAG System 🏆" if rag_total > llm_total else "LLM-Only 🏆" if llm_total > rag_total else "Tie 🤝"

print(f"""
📊 System Performance Summary:

════════════════════════════════════════════════════════════════
RAG SYSTEM (Hybrid Search + LLM):
════════════════════════════════════════════════════════════════
Total samples: {len(df_metrics)}

Translation Quality:
  • BLEU:         {avg_metrics['BLEU']:.2f}%
  • METEOR:       {avg_metrics['METEOR']:.2f}%
  • chrF:         {avg_metrics['chrF']:.2f}%
  • ROUGE-1:      {avg_metrics['ROUGE-1']:.2f}%
  • Exact Match:  {avg_metrics['Exact Match']:.2f}%

Retrieval Quality:
  • Recall@1:     {avg_metrics['Recall@1']:.2f}%
  • Recall@10:    {avg_metrics['Recall@10']:.2f}%
  • MRR:          {avg_metrics['MRR']:.2f}%
  • Avg Score:    {avg_metrics['Avg Retrieval Score']:.2f}%

════════════════════════════════════════════════════════════════
LLM-ONLY (No RAG):
════════════════════════════════════════════════════════════════
Total samples: {len(df_llm_only_metrics)}

Translation Quality:
  • BLEU:         {llm_only_averages['BLEU']:.2f}%
  • METEOR:       {llm_only_averages['METEOR']:.2f}%
  • chrF:         {llm_only_averages['chrF']:.2f}%
  • ROUGE-1:      {llm_only_averages['ROUGE-1']:.2f}%
  • Exact Match:  {llm_only_averages['Exact Match']:.2f}%

════════════════════════════════════════════════════════════════
COMPARISON RESULTS:
════════════════════════════════════════════════════════════════
🏆 Overall Winner: {overall_winner}

Translation Improvement (RAG - LLM):
  • BLEU:    {avg_metrics['BLEU'] - llm_only_averages['BLEU']:+.2f}%
  • METEOR:  {avg_metrics['METEOR'] - llm_only_averages['METEOR']:+.2f}%
  • chrF:    {avg_metrics['chrF'] - llm_only_averages['chrF']:+.2f}%

Sample-wise Wins:
  • RAG wins:      {wins['RAG']} ({wins['RAG']/total_comparisons*100:.1f}%)
  • LLM-only wins: {wins['LLM']} ({wins['LLM']/total_comparisons*100:.1f}%)

Retrieval Effectiveness:
  • Recall@10:     {avg_metrics['Recall@10']:.1f}% (RAG finds relevant in top 10)
  • MRR:           {avg_metrics['MRR']:.1f}% (Average rank quality)

════════════════════════════════════════════════════════════════
CONCLUSIONS:
════════════════════════════════════════════════════════════════
{"✅ The RAG system significantly outperforms LLM-only approach." if avg_metrics['BLEU'] > llm_only_averages['BLEU'] + 5 else
 "🟡 The RAG system shows moderate improvement over LLM-only." if avg_metrics['BLEU'] > llm_only_averages['BLEU'] else
 "❌ The LLM-only approach performs comparably or better than RAG."}

{"✅ Retrieval quality is good (Recall@10 > 40%)." if avg_metrics['Recall@10'] > 40 else
 "🟡 Retrieval quality is moderate (Recall@10 20-40%)." if avg_metrics['Recall@10'] > 20 else
 "❌ Retrieval quality needs improvement (Recall@10 < 20%)."}

📁 Output Files:
1. rag_comprehensive_results.csv - RAG results with all metrics
2. llm_only_comprehensive_results.csv - LLM-only results with all metrics
3. complete_comparison_summary.csv - Full comparison table
4. comparison_statistics.csv - Detailed statistical analysis
""")

print("="*70)
print("\n🎉 Complete evaluation finished!")
print("="*70 + "\n")



🏆 SAMPLE-BY-SAMPLE WIN/LOSS ANALYSIS

BLEU Wins:
  RAG:       19 ( 76.0%)
  LLM-Only:   4 ( 16.0%)
  Tie:        2 (  8.0%)

METEOR Wins:
  RAG:       22 ( 88.0%)
  LLM-Only:   1 (  4.0%)
  Tie:        2 (  8.0%)

CHRF Wins:
  RAG:       21 ( 84.0%)
  LLM-Only:   4 ( 16.0%)
  Tie:        0 (  0.0%)

──────────────────────────────────────────────────────────────────────
Overall Wins (across all metrics):
  🏆 RAG:       62 ( 82.7%)
  🏆 LLM-Only:   9 ( 12.0%)
  🤝 Tie:        4 (  5.3%)

💾 SAVING ALL RESULTS
✅ RAG comprehensive results saved to: rag_comprehensive_results.csv
✅ LLM-only comprehensive results saved to: llm_only_comprehensive_results.csv
✅ Comparison summary saved to: complete_comparison_summary.csv
✅ Statistics saved to: comparison_statistics.csv


✅ FINAL COMPREHENSIVE EVALUATION REPORT

📊 System Performance Summary:

════════════════════════════════════════════════════════════════
RAG SYSTEM (Hybrid Search + LLM):
══════════════════════════════════════════════════════════